# 데이터 확인

In [9]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import config 
from config import DB_CONFIG
import pymysql  

In [7]:
# MySQL 연결
conn = pymysql.connect(
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    database=DB_CONFIG['database'],
    port=DB_CONFIG['port']
)

car_siren = f"SELECT * FROM merge50_car_siren"
car_horn = f"SELECT * FROM merge50_car_horn"
car_driving = f'SELECT * FROM merge50_car_driving'
motorcycle_horn  = f'SELECT * FROM merge50_motorcycle_horn'
motorcycle_driving  = f'SELECT * FROM merge50_motorcycle_driving'

car_siren_df =  pd.read_sql(car_siren, conn)
car_horn_df = pd.read_sql(car_horn, conn) 
car_driving_df = pd.read_sql(car_driving, conn) 
motorcycle_horn_df= pd.read_sql(motorcycle_horn, conn) 
motorcycle_driving_df = pd.read_sql(motorcycle_driving, conn) 

conn.close()

In [8]:
print(f'car_siren 데이터 개수: {car_siren_df.shape[0]}')
print(f'car_horn 데이터 개수: {car_horn_df.shape[0]}')
print(f'car_driving 데이터 개수: {car_driving_df.shape[0]}')
print(f'motorcycle_horn 데이터 개수: {motorcycle_horn_df.shape[0]}')
print(f'motorcycle_driving 데이터 개수: {motorcycle_driving_df.shape[0]}')

car_siren 데이터 개수: 1990
car_horn 데이터 개수: 3189
car_driving 데이터 개수: 1682
motorcycle_horn 데이터 개수: 4560
motorcycle_driving 데이터 개수: 4735


In [8]:
car_siren_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 45 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fileName       1990 non-null   object 
 1   labelName      1990 non-null   object 
 2   area_start     1990 non-null   float64
 3   area_end       1990 non-null   float64
 4   category_01    1990 non-null   object 
 5   category_02    1990 non-null   object 
 6   category_03    1990 non-null   object 
 7   decibel        1990 non-null   int64  
 8   soundQuality   1990 non-null   object 
 9   subCategory    1990 non-null   object 
 10  bitRate        1990 non-null   object 
 11  duration       1990 non-null   float64
 12  fileFormat     1990 non-null   object 
 13  fileSize       1990 non-null   int64  
 14  recodingType   1990 non-null   object 
 15  sampleRate     1990 non-null   object 
 16  acqDevice      1990 non-null   object 
 17  acqMethod      1990 non-null   object 
 18  acqType 

In [7]:
car_horn_df.iloc[ : ,-13:] 

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,-493.29100,48.543324,11.231160,41.547720,18.616114,32.907097,1.259547,7.328120,16.237625,12.434523,-16.157902,1.341950,9.543132
1,-328.99643,110.927765,13.474552,37.183453,-10.836115,25.585962,0.232516,5.053774,12.176615,4.831705,-8.831820,10.174751,2.365335
2,-404.03650,30.101658,0.969419,49.188316,10.002414,34.354510,-9.145320,4.781949,6.678245,14.459094,-22.123974,-0.130046,9.682464
3,-380.94937,82.016785,-38.369390,10.472818,-25.531940,-0.819976,-21.547447,16.138535,-0.490012,0.726548,-9.243734,-1.005557,-10.407813
4,-251.55934,173.891480,-30.198183,45.578636,8.823283,17.660015,-4.340397,10.337170,4.226047,1.150458,-1.714232,2.508800,-1.459536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3184,-369.49686,134.442350,55.971985,22.243414,22.741440,17.390629,3.979185,0.757772,5.928487,1.002731,-12.015182,-11.394083,0.522755
3185,-426.90427,130.722400,47.745327,8.306962,5.675391,15.449170,5.677291,6.208929,-0.195498,-5.397255,-2.674880,-6.652485,-10.272819
3186,-316.11063,194.271060,-29.530697,35.808052,12.489090,23.182838,11.050718,3.705868,11.450807,-0.165664,2.353512,0.390641,5.358946
3187,-228.09323,64.343980,-13.663012,41.246460,-7.827433,21.293640,-13.415158,3.358162,5.326135,-3.165687,-11.261456,-1.512664,6.411310


In [6]:
#데이터프레임 병합 
df_combined = pd.concat([car_horn_df, car_siren_df,car_driving_df,motorcycle_horn_df,motorcycle_driving_df], ignore_index=True)

print(f'df_combined 행 개수: {len(df_combined)}') 
df_combined.head()

df_combined 행 개수: 16156


,fileName,labelName,area_start,area_end,category_01,category_02,category_03,decibel,soundQuality,subCategory,...,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,1.car_horn_30849_1.wav,1.car_horn_30849_1.wav,2.0,11.048,교통소음,자동차,차량경적,68,정상,소형차경적,...,37.183453,-10.836115,25.585962,0.232516,5.053774,12.176615,4.831705,-8.831820,10.174751,2.365335
1,1.car_horn_30865_1.wav,1.car_horn_30865_1.wav,2.0,10.672,교통소음,자동차,차량경적,75,정상,소형차경적,...,10.472818,-25.531940,-0.819976,-21.547447,16.138535,-0.490012,0.726548,-9.243734,-1.005557,-10.407813
2,1.car_horn_30869_1.wav,1.car_horn_30869_1.wav,2.0,12.245,교통소음,자동차,차량경적,68,정상,소형차경적,...,-2.516794,-12.093688,21.846487,10.806868,14.971516,9.503950,15.538783,15.781241,0.750223,-8.508451
3,1.car_horn_30874_1.wav,1.car_horn_30874_1.wav,2.0,10.289,교통소음,자동차,차량경적,69,정상,소형차경적,...,18.060278,-17.298338,28.503597,14.064431,7.452422,12.441176,16.612614,2.910048,-6.256705,-6.246398
4,1.car_horn_59_1.wav,1.car_horn_59_1.wav,2.0,4.029,교통소음,자동차,차량경적,107,정상,소형차경적,...,21.661055,-9.801945,12.513287,2.843365,8.194932,7.828580,8.889499,5.067267,2.928293,10.400720


# lightGBM + ODD

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# 레이블(label)과 피처(features) 분리
X = df_combined.iloc[:, -13:]  # 마지막 13개 컬럼을 특성(MFCC)으로 사용
y = df_combined['category_03'].astype('category').cat.codes  # 범주형 데이터를 숫자로 변환

# 범주형 데이터(레이블) 숫자로 변환
y = y.astype('category').cat.codes

# 클래스 레이블 매핑
class_labels = df_combined['category_03'].astype('category').cat.categories

# 데이터셋 분할 (train: 80%, test: 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_classes = len(set(y))
print(num_classes)

# LightGBM 모델 정의 및 학습
model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# 평가 결과 출력
acc = accuracy_score(y_test, y_pred)
print(f"LightGBM 정확도: {acc:.4f}")

# 혼동 행렬 및 분류 보고서 출력
print("\nConfusion Matrix:")
conf_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), index=class_labels, columns=class_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels.tolist()))


5
LightGBM 정확도: 0.9208

Confusion Matrix:
        이륜차경적  이륜차주행음  차량경적  차량사이렌  차량주행음
이륜차경적     892       5    52      0      0
이륜차주행음      3     859    16      3     34
차량경적       26      25   553      5     12
차량사이렌       0       4     7    408      3
차량주행음       0      51    10      0    264

Classification Report:
              precision    recall  f1-score   support

       이륜차경적       0.97      0.94      0.95       949
      이륜차주행음       0.91      0.94      0.92       915
        차량경적       0.87      0.89      0.88       621
       차량사이렌       0.98      0.97      0.97       422
       차량주행음       0.84      0.81      0.83       325

    accuracy                           0.92      3232
   macro avg       0.91      0.91      0.91      3232
weighted avg       0.92      0.92      0.92      3232



In [10]:
import lightgbm as lgb

# 모델 학습
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

# 모델 저장 (Booster 객체 사용)
model.booster_.save_model('lgbm_model.txt')

print("모델이 성공적으로 저장되었습니다!")

모델이 성공적으로 저장되었습니다!


In [11]:
import os
import json
import librosa
import numpy as np
import pandas as pd

data_dir = "/home/ubuntu/data/etc_noise_data_test"

# 기존 데이터프레임이 존재하는지 확인
try:
    df_combined
except NameError:
    df_combined = pd.DataFrame()  # 없으면 빈 데이터프레임 생성

json_data = []
for file in os.listdir(data_dir):
    if file.endswith(".json"):
        with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
            data = json.load(f)

            annotations = data.get("annotations", [])
            annotation = annotations[0] if annotations and isinstance(annotations, list) else {}

            # JSON 파일명에서 확장자 제거
            base_name = os.path.splitext(file)[0]

            json_data.append({
                "baseName": base_name,  # 확장자 없는 파일명 저장
                "fileName": data["audio"]["fileName"],
                "labelName": annotation.get("labelName", None),
                "area_start": annotation.get("area", {}).get("start", None),
                "area_end": annotation.get("area", {}).get("end", None),
                "category_01": annotation.get("categories", {}).get("category_01", None),
                "category_02": annotation.get("categories", {}).get("category_02", None),
                "category_03": annotation.get("categories", {}).get("category_03", None),
                "decibel": annotation.get("decibel", None),
                "soundQuality": annotation.get("soundQuality", None),
                "subCategory": annotation.get("subCategory", None),
            })

df_json = pd.DataFrame(json_data)

# WAV 파일에서 MFCC 추출하는 함수
def extract_mfcc(file_path, sr=22050, n_mfcc=13):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)  # MFCC 평균값 반환
    except Exception as e:
        print(f"오류 발생: {file_path} - {e}")
        return [None] * n_mfcc  # 오류 발생 시 None 리스트 반환

wav_data = []
for file in os.listdir(data_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(data_dir, file)
        base_name = "_".join(file.split("_")[:-1])  # "_1.wav" 부분 제거하고 기본 파일명 추출
        mfcc_features = extract_mfcc(file_path)

        wav_data.append({
            "baseName": base_name,  # JSON과 매칭할 기본 파일명 저장
            **{f"mfcc_{i+1}": mfcc for i, mfcc in enumerate(mfcc_features)}
        })

df_wav = pd.DataFrame(wav_data)

# # JSON 데이터와 WAV 데이터 병합 (baseName 기준)
df_etc = pd.merge(df_json, df_wav, on="baseName", how="left").drop(columns=["baseName"])

# # 기존 데이터프레임 df_combined에 추가
df_combined2 = pd.concat([df_combined, df_etc], ignore_index=True)

# # 결과 확인
# print(df_combined.head())

In [12]:
df_combined2.shape

(16176, 45)

In [13]:
# 📌 4️⃣ OOD 데이터 로드 & 전처리 (MFCC 특징 사용)
ood_X_test = df_etc.iloc[:, -13:]  # OOD 데이터의 MFCC 13개 컬럼 사용

# 📌 5️⃣ Softmax 기반 OOD 탐지 함수
def predict_with_ood_detection(model, X, threshold=0.6):
    probs = model.predict_proba(X)  # Softmax 확률 출력
    max_probs = np.max(probs, axis=1)  # 가장 높은 확률 값
    preds = np.argmax(probs, axis=1)  # 가장 높은 확률의 클래스

    # 특정 확률(threshold) 이하이면 "기타 소음"(-1)으로 변경
    final_preds = np.where(max_probs < threshold, -1, preds)
    
    return final_preds

# 📌 6️⃣ OOD 탐지 적용 (테스트 데이터 & 기타 소음 데이터)
y_pred_test = predict_with_ood_detection(model, X_test, threshold=0.5)
y_pred_ood = predict_with_ood_detection(model, ood_X_test, threshold=0.5)

# 📌 7️⃣ 결과 출력
print("✅ 테스트 데이터 예측 결과:", y_pred_test)
print("✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링):", y_pred_ood)

✅ 테스트 데이터 예측 결과: [2 4 4 ... 4 0 3]
✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링): [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 4 0]


In [14]:
df_etc.shape

(20, 23)

In [15]:
df_etc.columns

Index(['fileName', 'labelName', 'area_start', 'area_end', 'category_01',
       'category_02', 'category_03', 'decibel', 'soundQuality', 'subCategory',
       'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
       'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13'],
      dtype='object')

In [16]:
df_combined.shape

(16156, 45)

In [17]:
df_combined.columns

Index(['fileName', 'labelName', 'area_start', 'area_end', 'category_01',
       'category_02', 'category_03', 'decibel', 'soundQuality', 'subCategory',
       'bitRate', 'duration', 'fileFormat', 'fileSize', 'recodingType',
       'sampleRate', 'acqDevice', 'acqMethod', 'acqType', 'areaUse',
       'dayNight', 'direction', 'distance', 'district', 'latitude',
       'longitude', 'micClass', 'obstacle', 'place', 'recordingTime', 'urban',
       'weather', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6',
       'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12',
       'mfcc_13'],
      dtype='object')

# df_combined2 (df_combined + df_etc) 모델 생성 후 성능 체크

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 레이블(label)과 피처(features) 분리
X = df_combined2.iloc[:, -13:]  # 마지막 13개 컬럼을 특성(MFCC)으로 사용
y = df_combined2['category_03'].astype('category').cat.codes  # 범주형 데이터를 숫자로 변환

# 범주형 데이터(레이블) 숫자로 변환
y = y.astype('category').cat.codes

# 클래스 레이블 매핑
class_labels = df_combined2['category_03'].astype('category').cat.categories

# 데이터셋 분할 (train: 80%, test: 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_classes = len(set(y))
print(num_classes)
    
# LightGBM 모델 정의 및 학습
model2 = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)

# 평가 결과 출력
acc = accuracy_score(y_test, y_pred)
print(f"LightGBM 정확도: {acc:.4f}")

# 혼동 행렬 및 분류 보고서 출력
print("\nConfusion Matrix:")
conf_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), index=class_labels, columns=class_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels.tolist()))


9
LightGBM 정확도: 0.7361

Confusion Matrix:
        기차  비행기  이륜차경적  이륜차주행음  지하철  차량경적  차량사이렌  차량주행음  헬리콥터
기차       0    0      0       0    0     0      0      0     0
비행기      0    0      0       1    0     0      0      1     0
이륜차경적    2    3    777      57    0    84      8     17     1
이륜차주행음   0    5     15     758   10    63     15     52     2
지하철      0    0      0       1    0     0      0      0     0
차량경적     1    2     62     111    2   410      6     27     0
차량사이렌    0    0      6      26    1    33    331     16     0
차량주행음    0    0     15     170    4    29      5    106     0
헬리콥터     0    0      0       1    0     0      0      0     0

Classification Report:
              precision    recall  f1-score   support

          기차       0.00      0.00      0.00         0
         비행기       0.00      0.00      0.00         2
       이륜차경적       0.89      0.82      0.85       949
      이륜차주행음       0.67      0.82      0.74       920
         지하철       0.00      0.00      0.00

/home/ubuntu/anaconda3/envs/ml_env_python3.6/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/anaconda3/envs/ml_env_python3.6/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/anaconda3/envs/ml_env_python3.6/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [19]:
# 📌 4️⃣ OOD 데이터 로드 & 전처리 (MFCC 특징 사용)
ood_X_test = df_combined2.iloc[:, -13:]  # OOD 데이터의 MFCC 13개 컬럼 사용

# 📌 5️⃣ Softmax 기반 OOD 탐지 함수
def predict_with_ood_detection(model2, X, threshold=0.6):
    probs = model2.predict_proba(X)  # Softmax 확률 출력
    max_probs = np.max(probs, axis=1)  # 가장 높은 확률 값
    preds = np.argmax(probs, axis=1)  # 가장 높은 확률의 클래스

    # 특정 확률(threshold) 이하이면 "기타 소음"(-1)으로 변경
    final_preds = np.where(max_probs < threshold, -1, preds)
    
    return final_preds

# 📌 6️⃣ OOD 탐지 적용 (테스트 데이터 & 기타 소음 데이터)
#y_pred_test = predict_with_ood_detection(model, X_test, threshold=0.8)
y_pred_ood = predict_with_ood_detection(model2, ood_X_test, threshold=0.8)

# 📌 7️⃣ 결과 출력
#print("✅ 테스트 데이터 예측 결과:", y_pred_test)
print("✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링):", y_pred_ood)

✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링): [ 5  2  5 ... -1 -1  0]


In [20]:
import numpy as np

# NumPy 배열 출력 옵션 변경 (생략 없이 출력)
np.set_printoptions(threshold=np.inf)

print("✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링):", y_pred_ood)

✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링): [ 5  2  5  5 -1  7  5  5 -1  5  0  5  5  5  5  5  2  2  5  2  5  5  5  2
  5  3  5  5  7 -1  5  3  5  5  5 -1  2  3 -1  3  5  3  5  5  5  5  2  5
  5  5  5  5  5  2  2  3  3  3  5  3  5  5  7  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5 -1 -1  5  5  3  2  3
  3  3  5  5  5  5  2  2  5 -1  7 -1  5 -1  3 -1  5  2  5 -1 -1  5  5  5
  5  5  5  5  2  5  5  5  5  2  2  2  2  2  2  3  2  5  8  5  5  5  5  2
  5  5  8  5  5  3  5  5  5 -1  5  5  5  5  5  5  5  5  2  5  5  5  5  5
  5  5  5  5  5  5  5  5  2  5  5  3  5  5 -1 -1  7  7 -1 -1 -1  5  2  5
  5  5  5  5  5  5  5  5  3  5 -1  5  5  5  5  5  5  3  5  5  5  5  5  5
  5  5  5  3  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  2  5  5  5  5  5  5  5  5  5  5  6  3  5  3  3  5
  3  5  5  5  5  5  3  5  5  5  5  5  5  3  3  5  5  5  5  3  5  5  5  5
  5  6  5  5  5 -1  5  5  5  5  5  5  5  5  5  5  5  5  5  5  3  3  5  5
  5  5  5  5 